## Part 1: Define metrics

In [ ]:
from urllib.parse import urlparse, parse_qs

def parse_url(url):
    """
    Parse a URL into its components.
    """
    parsed_url = urlparse(url)
    query_params = parse_qs(parsed_url.query)
    return {
        'path': parsed_url.path,
        'params': query_params
    }

def is_url_structure_matching(candidate, reference):
    """
    Compare the path and query parameters of the candidate and reference URLs.
    """
    if candidate['path'] != reference['path']:
        return False

    if sorted(candidate['params'].keys()) != sorted(reference['params'].keys()):
        return False

    for key in reference['params']:
        if key not in candidate['params']:
            return False
        if sorted(candidate['params'][key]) != sorted(reference['params'][key]):
            return False
    
    return True

def evaluate_get_request_accuracy(generated_url, reference_url):
    """
    Evaluate if the generated GET request is equivalent to the reference GET request.
    """
    candidate = parse_url(generated_url)
    reference = parse_url(reference_url)
    
    return is_url_structure_matching(candidate, reference)

def score_ast_batched(preds, refs):
    evals = tuple(map(evaluate_get_request_accuracy, preds, refs))
    return sum(evals) / len(evals)

In [ ]:
import evaluate
bert_scorer = evaluate.load("bertscore")
bert_score_fn = lambda preds, refs: bert_scorer.compute(predictions=preds, references=refs, lang="en", model_type="microsoft/codebert-base", num_layers=12, device="cuda")

In [ ]:
preds = ["/v3/query/?q=symbol:ZFAND4&species=mouse"]
refs = ["/v3/query/?species=mouse&q=symbol:ZFAND4"]

print("AST eval", score_ast_batched(preds, refs))
print("BERT Score", bert_score_fn(preds, refs))

## Part 2: Load models

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
with open("gene_query_docs.txt", "r") as doc_fd:
    docs = doc_fd.read()

with open("data/original/compact_desc_with_context.csv") as desc_fd:
    description = desc_fd.read()

In [ ]:
import inspect
prompt_template = inspect.cleandoc("""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
Use the documentation and schema to complete the user-given task.
Docs: {docs}\n Schema: {description}<|eot_id|><|start_header_id|>user<|end_header_id|>
{instruction}. Write an API call.
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
""")

In [ ]:
prompt_gen = lambda inst: prompt_template.format(docs=docs, description=description, instruction=inst)
prompt = prompt_gen("Find the UniProt ID for the ENSG00000103187 gene in human. Limit the search to Ensembl gene IDs.")
print("Start", prompt[:250])
print("End", prompt[-150:])

In [ ]:
import outlines

model = AutoModelForCausalLM.from_pretrained("models/meta_llama3_1", torch_dtype=torch.bfloat16, output_attentions=True).to("cuda")
model.eval()
tokenizer = AutoTokenizer.from_pretrained("models/meta_llama3_1")

In [ ]:
def evaluate(api_call: str):
    return None

model = outlines.models.Transformers(model, tokenizer)
generator = outlines.generate.json(model, evaluate)
# generator = outlines.generate.regex(model, r"/v3/.+/.+")

In [ ]:
sample_api_call = generator([prompt])
sample_api_call

## Part 3: Evaluate

In [ ]:
from datasets import load_dataset

dataset = load_dataset("moltres23/biothings-query-instruction-pairs", split="test")
dataset

In [ ]:
import tqdm

BATCH_SIZE = 1
all_responses = []
prompt_gen = lambda inst: prompt_template.format(docs=docs, description=description, instruction=inst)
with torch.no_grad():
    for idx in tqdm.tqdm(range(0, len(dataset), BATCH_SIZE)):
        batch = dataset[idx:(idx + BATCH_SIZE)]
        batched_inputs = list(map(prompt_gen, batch["instruction"]))
        batch_responses = generator(batched_inputs)
        all_responses.extend(batch_responses.values())

In [ ]:
import pickle

In [ ]:
with open('responses.pkl', 'wb') as fd:
   pickle.dump(all_responses, fd)

In [ ]:
with open('responses.pkl', 'rb') as fd:
   all_responses = pickle.load(fd)

In [ ]:
import re
def regex_match(string):
    match = re.search(r"/v3/.+", string)
    return match.group(0) if match is not None else ""

In [ ]:
regex_match("GET https://mygene.infov3/query?fields=human")

In [ ]:
all_responses = list(map(lambda x: regex_match(x), all_responses))
ast_eval = score_ast_batched(all_responses, dataset["output"])
bertscore_evals = bert_score_fn(all_responses, dataset["output"])

In [ ]:
import numpy as np
import scipy.stats

# we include 0s in mean because otherwise merely getting
# one correct answer will skew the metric
print("AST eval", ast_eval)
print("BERT Score", np.mean(bertscore_evals["recall"]))

In [ ]:
all_responses[:10]

In [ ]:
dataset["output"][:10]